importing libraries and other utilities 

In [ ]:
import sklearn
import joblib
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from haversine import haversine,Unit
from user_location import current_location_user
from search_location import search_location
from 





making the data frame
